In [1]:
#Import Dependencies
import pandas as pd
import numpy as np

#Import matplotlib dependencies
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress

import json 

import geopandas as gpd

#File to load
file_path = "./Data/minwage.csv"

In [2]:
#Create dataframe
min_wage_df = pd.read_csv(file_path,encoding='cp1252') 
min_wage_df.head()


,Year,State,State.Minimum.Wage,State.Minimum.Wage.2020.Dollars,Federal.Minimum.Wage,Federal.Minimum.Wage.2020.Dollars,Effective.Minimum.Wage,Effective.Minimum.Wage.2020.Dollars,CPI.Average,Department.Of.Labor.Uncleaned.Data,Department.Of.Labor.Cleaned.Low.Value,Department.Of.Labor.Cleaned.Low.Value.2020.Dollars,Department.Of.Labor.Cleaned.High.Value,Department.Of.Labor.Cleaned.High.Value.2020.Dollars,Footnote
0,1968,Alabama,0.00000,0.00,1.15,8.55,1.15,8.55,34.8,...,0.00000,0.00,0.00000,0.00,NaN
1,1968,Alaska,2.10000,15.61,1.15,8.55,2.10,15.61,34.8,2.1,2.10000,15.61,2.10000,15.61,NaN
2,1968,Arizona,0.46800,3.48,1.15,8.55,1.15,8.55,34.8,18.72 - 26.40/wk(b),0.46800,3.48,0.66000,4.91,(b)
3,1968,Arkansas,0.15625,1.16,1.15,8.55,1.15,8.55,34.8,1.25/day(b),0.15625,1.16,0.15625,1.16,(b)
4,1968,California,1.65000,12.26,1.15,8.55,1.65,12.26,34.8,1.65(b),1.65000,12.26,1.65000,12.26,(b)


In [3]:
min_wage_df['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'U.S. Virgin Islands',
       'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia',
       'Wisconsin', 'Wyoming'], dtype=object)

In [4]:
# Show column names 
for col in min_wage_df.columns: 
    print(col) 

Year
State
State.Minimum.Wage
State.Minimum.Wage.2020.Dollars
Federal.Minimum.Wage
Federal.Minimum.Wage.2020.Dollars
Effective.Minimum.Wage
Effective.Minimum.Wage.2020.Dollars
CPI.Average
Department.Of.Labor.Uncleaned.Data
Department.Of.Labor.Cleaned.Low.Value
Department.Of.Labor.Cleaned.Low.Value.2020.Dollars
Department.Of.Labor.Cleaned.High.Value
Department.Of.Labor.Cleaned.High.Value.2020.Dollars
Footnote


In [5]:
clean_min_wage_df = min_wage_df[['Year', 'State', 'State.Minimum.Wage', 'Federal.Minimum.Wage']]
clean_min_wage_df.head(10)

,Year,State,State.Minimum.Wage,Federal.Minimum.Wage
0,1968,Alabama,0.00000,1.15
1,1968,Alaska,2.10000,1.15
2,1968,Arizona,0.46800,1.15
3,1968,Arkansas,0.15625,1.15
4,1968,California,1.65000,1.15
5,1968,Colorado,1.00000,1.15
6,1968,Connecticut,1.40000,1.15
7,1968,Delaware,1.25000,1.15
8,1968,District of Columbia,1.25000,1.15
9,1968,Florida,0.00000,1.15


In [6]:
# Drop earlier years

drop_early_df = clean_min_wage_df[clean_min_wage_df['Year'] >= 1970]
drop_early_df.head(10)


,Year,State,State.Minimum.Wage,Federal.Minimum.Wage
108,1970,Alabama,0.000,1.3
109,1970,Alaska,2.100,1.3
110,1970,Arizona,0.468,1.3
111,1970,Arkansas,1.100,1.3
112,1970,California,1.650,1.3
113,1970,Colorado,1.000,1.3
114,1970,Connecticut,1.600,1.3
115,1970,Delaware,1.250,1.3
116,1970,District of Columbia,1.600,1.3
117,1970,Florida,0.000,1.3


In [7]:
#Load in Latlngs
latlong_file_path = "./Data/statelatlong.csv"
latlong_df = pd.read_csv(latlong_file_path,encoding='cp1252') 
latlong_df = latlong_df.drop('State', 1)
latlong_df = latlong_df.rename(columns={'City': 'State'})
latlong_df.head()

,Latitude,Longitude,State
0,32.601011,-86.680736,Alabama
1,61.302501,-158.775020,Alaska
2,34.168219,-111.930907,Arizona
3,34.751928,-92.131378,Arkansas
4,37.271875,-119.270415,California


In [8]:
latlong_df['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [9]:
minwage_with_latlng = pd.merge(drop_early_df,  
                     latlong_df,  
                     on ='State')

minwage_with_latlng

,Year,State,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude
0,1970,Alabama,0.00,1.30,32.601011,-86.680736
1,1971,Alabama,0.00,1.30,32.601011,-86.680736
2,1972,Alabama,0.00,1.60,32.601011,-86.680736
3,1973,Alabama,0.00,1.60,32.601011,-86.680736
4,1974,Alabama,0.00,1.60,32.601011,-86.680736
...,...,...,...,...,...,...
2596,2016,Wyoming,5.15,7.25,43.000325,-107.554567
2597,2017,Wyoming,5.15,7.25,43.000325,-107.554567
2598,2018,Wyoming,5.15,7.25,43.000325,-107.554567
2599,2019,Wyoming,5.15,7.25,43.000325,-107.554567


In [10]:
len(set(minwage_with_latlng['State'].values))

51

In [11]:
print(minwage_with_latlng["Year"].max())
print(minwage_with_latlng['Year'].min())

2020
1970


In [12]:
# Create bins in which to place values based upon TED Talk views
bins = [1969, 1979, 1989, 1999, 2009, 2019,2020 ]

# Create labels for these bins
group_labels = ['1970',"1980", "1990", "2000", "2010",'2020']

In [13]:
# Place the data series into a new column inside of the DataFrame
minwage_with_latlng["Decade"] = pd.cut(minwage_with_latlng["Year"], bins, labels=group_labels)
minwage_with_latlng.sample(20)

,Year,State,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude,Decade
1519,2010,New Hampshire,7.25,7.25,44.001231,-71.579923,2010
1461,2003,Nevada,5.15,5.15,38.502032,-117.023060,2000
1155,2003,Michigan,5.15,5.15,44.943560,-86.415805,2000
829,1983,Kansas,1.60,3.35,38.498779,-98.320078,1980
572,1981,Hawaii,3.10,3.35,20.460000,-157.505000,1980
2314,1989,Vermont,3.55,3.35,43.871755,-72.447783,1980
13,1983,Alabama,0.00,3.35,32.601011,-86.680736,1980
336,2000,Connecticut,6.15,5.15,41.518784,-72.757507,2000
1309,2004,Missouri,5.15,5.15,38.304662,-92.437099,2000
1079,1978,Massachusetts,2.10,2.20,42.062940,-71.718067,1970


In [14]:
# Create a GroupBy object based upon "View Group"


statistics_df=round(minwage_with_latlng.groupby(['Decade', 'State']).agg(['mean', 'median'])['State.Minimum.Wage'],2)
statistics_df=statistics_df.reset_index()
statistics_df

,Decade,State,mean,median
0,1970,Alabama,0.00,0.00
1,1970,Alaska,2.44,2.10
2,1970,Arizona,0.28,0.47
3,1970,Arkansas,1.50,1.20
4,1970,California,1.88,1.65
...,...,...,...,...
301,2020,Virginia,7.25,7.25
302,2020,Washington,13.50,13.50
303,2020,West Virginia,8.75,8.75
304,2020,Wisconsin,7.25,7.25


In [15]:
statistics_df_federal=round(minwage_with_latlng.groupby(['Decade', 'State']).agg(['mean', 'median'])['Federal.Minimum.Wage'],2)
statistics_df_federal=statistics_df_federal.reset_index()
statistics_df_federal

,Decade,State,mean,median
0,1970,Alabama,1.85,1.60
1,1970,Alaska,1.85,1.60
2,1970,Arizona,1.85,1.60
3,1970,Arkansas,1.85,1.60
4,1970,California,1.85,1.60
...,...,...,...,...
301,2020,Virginia,7.25,7.25
302,2020,Washington,7.25,7.25
303,2020,West Virginia,7.25,7.25
304,2020,Wisconsin,7.25,7.25


In [16]:
statistics_df

,Decade,State,mean,median
0,1970,Alabama,0.00,0.00
1,1970,Alaska,2.44,2.10
2,1970,Arizona,0.28,0.47
3,1970,Arkansas,1.50,1.20
4,1970,California,1.88,1.65
...,...,...,...,...
301,2020,Virginia,7.25,7.25
302,2020,Washington,13.50,13.50
303,2020,West Virginia,8.75,8.75
304,2020,Wisconsin,7.25,7.25


In [17]:
minwage_with_latlng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2601 entries, 0 to 2600
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Year                  2601 non-null   int64   
 1   State                 2601 non-null   object  
 2   State.Minimum.Wage    2601 non-null   float64 
 3   Federal.Minimum.Wage  2601 non-null   float64 
 4   Latitude              2601 non-null   float64 
 5   Longitude             2601 non-null   float64 
 6   Decade                2601 non-null   category
dtypes: category(1), float64(4), int64(1), object(1)
memory usage: 225.0+ KB


In [18]:
minwage_with_latlng['Decade']= minwage_with_latlng['Decade'].astype(str)
minwage_with_latlng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2601 entries, 0 to 2600
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  2601 non-null   int64  
 1   State                 2601 non-null   object 
 2   State.Minimum.Wage    2601 non-null   float64
 3   Federal.Minimum.Wage  2601 non-null   float64
 4   Latitude              2601 non-null   float64
 5   Longitude             2601 non-null   float64
 6   Decade                2601 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 242.6+ KB


In [19]:
#del minwage_with_latlng["Year"]

In [20]:
# print('We have {} geotagged rows'.format(len(minwage_with_latlng)))

In [21]:
# def df_to_geojson(minwage_with_latlng, properties, lat='Latitude', lon='Longitude'):
#     """
#     Turn a dataframe containing point data into a geojson formatted python dictionary
    
#     df : the dataframe to convert to geojson
#     properties : a list of columns in the dataframe to turn into geojson feature properties
#     lat : the name of the column in the dataframe that contains latitude data
#     lon : the name of the column in the dataframe that contains longitude data
#     """
    
#     # create a new python dict to contain our geojson data, using geojson format
#     geojson = {'type':'FeatureCollection', 'features':[]}

#     # loop through each row in the dataframe and convert each row to geojson format
#     for _, row in minwage_with_latlng.iterrows():
#         # create a feature template to fill in
#         feature = {'type':'Feature',
#                    'properties':{},
#                    'geometry':{'type':'Point',
#                                'coordinates':[]}}

#         # fill in the coordinates
#         feature['geometry']['coordinates'] = [row[lon],row[lat]]

#         # for each column, get the value and add it as a new feature property
#         for prop in properties:
#             feature['properties'][prop] = row[prop]
        
#         # add this feature (aka, converted dataframe row) to the list of features inside our dict
#         geojson['features'].append(feature)
    
#     return geojson

In [22]:
# useful_columns = ['Year','State','State.Minimum.Wage','State.Minimum.Wage.2020.Dollars','Federal.Minimum.Wage', 'Federal.Minimum.Wage.2020.Dollars','Latitude','Longitude','Decade']
# geojson_dict = df_to_geojson(minwage_with_latlng, properties=useful_columns)
# geojson_str = json.dumps(geojson_dict, indent=2)
# print(geojson_str)

In [23]:
# Can't use this, <class 'str'> type

In [24]:
geojson_internet=gpd.read_file('gz_2010_us_040_00_500k.json')
geojson_internet

,GEO_ID,STATE,NAME,LSAD,CENSUSAREA,geometry
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ..."
1,0400000US25,25,Massachusetts,,7800.058,"MULTIPOLYGON (((-70.83204 41.60650, -70.82373 ..."
2,0400000US26,26,Michigan,,56538.901,"MULTIPOLYGON (((-88.68443 48.11579, -88.67563 ..."
3,0400000US30,30,Montana,,145545.801,"POLYGON ((-104.05770 44.99743, -104.25015 44.9..."
4,0400000US32,32,Nevada,,109781.180,"POLYGON ((-114.05060 37.00040, -114.04999 36.9..."
5,0400000US34,34,New Jersey,,7354.220,"POLYGON ((-75.52684 39.65571, -75.52634 39.656..."
6,0400000US36,36,New York,,47126.399,"MULTIPOLYGON (((-71.94356 41.28668, -71.92680 ..."
7,0400000US37,37,North Carolina,,48617.905,"MULTIPOLYGON (((-82.60288 36.03983, -82.60074 ..."
8,0400000US39,39,Ohio,,40860.694,"MULTIPOLYGON (((-82.81349 41.72347, -82.81049 ..."
9,0400000US42,42,Pennsylvania,,44742.703,"POLYGON ((-75.41504 39.80179, -75.42804 39.809..."


In [25]:
renamed_df=geojson_internet.rename(columns = {'STATE':'Name', 'NAME':'State'})
renamed_df

,GEO_ID,Name,State,LSAD,CENSUSAREA,geometry
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ..."
1,0400000US25,25,Massachusetts,,7800.058,"MULTIPOLYGON (((-70.83204 41.60650, -70.82373 ..."
2,0400000US26,26,Michigan,,56538.901,"MULTIPOLYGON (((-88.68443 48.11579, -88.67563 ..."
3,0400000US30,30,Montana,,145545.801,"POLYGON ((-104.05770 44.99743, -104.25015 44.9..."
4,0400000US32,32,Nevada,,109781.180,"POLYGON ((-114.05060 37.00040, -114.04999 36.9..."
5,0400000US34,34,New Jersey,,7354.220,"POLYGON ((-75.52684 39.65571, -75.52634 39.656..."
6,0400000US36,36,New York,,47126.399,"MULTIPOLYGON (((-71.94356 41.28668, -71.92680 ..."
7,0400000US37,37,North Carolina,,48617.905,"MULTIPOLYGON (((-82.60288 36.03983, -82.60074 ..."
8,0400000US39,39,Ohio,,40860.694,"MULTIPOLYGON (((-82.81349 41.72347, -82.81049 ..."
9,0400000US42,42,Pennsylvania,,44742.703,"POLYGON ((-75.41504 39.80179, -75.42804 39.809..."


In [26]:
geojson_data=renamed_df.merge(minwage_with_latlng, on='State')
geojson_data

,GEO_ID,Name,State,LSAD,CENSUSAREA,geometry,Year,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude,Decade
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1970,1.60,1.30,45.218513,-69.014866,1970
1,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1971,1.60,1.30,45.218513,-69.014866,1970
2,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1972,1.40,1.60,45.218513,-69.014866,1970
3,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1973,1.40,1.60,45.218513,-69.014866,1970
4,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1974,1.40,1.60,45.218513,-69.014866,1970
...,...,...,...,...,...,...,...,...,...,...,...,...
2596,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2016,5.15,7.25,43.000325,-107.554567,2010
2597,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2017,5.15,7.25,43.000325,-107.554567,2010
2598,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2018,5.15,7.25,43.000325,-107.554567,2010
2599,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2019,5.15,7.25,43.000325,-107.554567,2010


In [27]:
geojson_data_final=geojson_data.merge(statistics_df, on=['Decade','State'])
geojson_data_final

,GEO_ID,Name,State,LSAD,CENSUSAREA,geometry,Year,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude,Decade,mean,median
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1970,1.60,1.30,45.218513,-69.014866,1970,1.86,1.60
1,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1971,1.60,1.30,45.218513,-69.014866,1970,1.86,1.60
2,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1972,1.40,1.60,45.218513,-69.014866,1970,1.86,1.60
3,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1973,1.40,1.60,45.218513,-69.014866,1970,1.86,1.60
4,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1974,1.40,1.60,45.218513,-69.014866,1970,1.86,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2596,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2016,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15
2597,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2017,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15
2598,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2018,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15
2599,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2019,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15


In [28]:
geojson_data_final=geojson_data_final.merge(statistics_df_federal, on=['Decade','State'])
geojson_data_final

,GEO_ID,Name,State,LSAD,CENSUSAREA,geometry,Year,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude,Decade,mean_x,median_x,mean_y,median_y
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1970,1.60,1.30,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
1,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1971,1.60,1.30,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
2,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1972,1.40,1.60,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
3,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1973,1.40,1.60,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
4,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1974,1.40,1.60,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2596,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2016,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2597,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2017,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2598,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2018,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2599,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2019,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25


In [29]:
geojson_data_to_upload = geojson_data_final.rename(columns = {'mean_x':'State_Average_by_Decade', 'median_x':'State_Median_by_Decade','mean_y':'Federal_Average_by_Decade', 'median_y':'Federal_Median_by_Decade'})
geojson_data_to_upload

,GEO_ID,Name,State,LSAD,CENSUSAREA,geometry,Year,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude,Decade,State_Average_by_Decade,State_Median_by_Decade,Federal_Average_by_Decade,Federal_Median_by_Decade
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1970,1.60,1.30,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
1,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1971,1.60,1.30,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
2,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1972,1.40,1.60,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
3,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1973,1.40,1.60,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
4,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",1974,1.40,1.60,45.218513,-69.014866,1970,1.86,1.60,1.85,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2596,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2016,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2597,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2017,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2598,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2018,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2599,0400000US56,56,Wyoming,,97093.141,"POLYGON ((-109.05008 41.00066, -109.17368 41.0...",2019,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25


In [30]:
# # rows = []

# for ii in range(1970, 2021):
#     test1 = geojson_data_to_upload[geojson_data_to_upload.Year == ii]

# # print(test1)
#     test1.to_file("./map/static/data/"+str(ii)+".geojson", driver='GeoJSON')
# # for ii , row in geojson_data_to_upload.iterrows():
# #     if(row.Year == 1970):
# #         rows.append(row)

In [32]:
print(type(geojson_data_to_upload))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [33]:
geojson_data_to_upload.to_file("minwage.geojson", driver='GeoJSON')

In [34]:
geojson_internet2=gpd.read_file('us_states.geojson')
geojson_internet2

,id,name,density,geometry
0,01,Alabama,94.650,"POLYGON ((-87.359 35.001, -85.607 34.985, -85...."
1,02,Alaska,1.264,"MULTIPOLYGON (((-131.602 55.118, -131.569 55.2..."
2,04,Arizona,57.050,"POLYGON ((-109.043 37.000, -109.048 31.332, -1..."
3,05,Arkansas,56.430,"POLYGON ((-94.474 36.502, -90.153 36.496, -90...."
4,06,California,241.700,"POLYGON ((-123.233 42.006, -122.379 42.012, -1..."
5,08,Colorado,49.330,"POLYGON ((-107.920 41.004, -105.729 40.998, -1..."
6,09,Connecticut,739.100,"POLYGON ((-73.054 42.039, -71.799 42.023, -71...."
7,10,Delaware,464.300,"POLYGON ((-75.414 39.804, -75.507 39.684, -75...."
8,11,District of Columbia,10065.000,"POLYGON ((-77.035 38.994, -76.909 38.895, -77...."
9,12,Florida,353.400,"POLYGON ((-85.497 30.998, -85.004 31.003, -84...."


In [35]:
renamed_df=geojson_internet2.rename(columns = {'name':'State'})
renamed_df

,id,State,density,geometry
0,01,Alabama,94.650,"POLYGON ((-87.359 35.001, -85.607 34.985, -85...."
1,02,Alaska,1.264,"MULTIPOLYGON (((-131.602 55.118, -131.569 55.2..."
2,04,Arizona,57.050,"POLYGON ((-109.043 37.000, -109.048 31.332, -1..."
3,05,Arkansas,56.430,"POLYGON ((-94.474 36.502, -90.153 36.496, -90...."
4,06,California,241.700,"POLYGON ((-123.233 42.006, -122.379 42.012, -1..."
5,08,Colorado,49.330,"POLYGON ((-107.920 41.004, -105.729 40.998, -1..."
6,09,Connecticut,739.100,"POLYGON ((-73.054 42.039, -71.799 42.023, -71...."
7,10,Delaware,464.300,"POLYGON ((-75.414 39.804, -75.507 39.684, -75...."
8,11,District of Columbia,10065.000,"POLYGON ((-77.035 38.994, -76.909 38.895, -77...."
9,12,Florida,353.400,"POLYGON ((-85.497 30.998, -85.004 31.003, -84...."


In [36]:
geojson_data=renamed_df.merge(minwage_with_latlng, on='State')
geojson_data

,id,State,density,geometry,Year,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude,Decade
0,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1970,0.00,1.30,32.601011,-86.680736,1970
1,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1971,0.00,1.30,32.601011,-86.680736,1970
2,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1972,0.00,1.60,32.601011,-86.680736,1970
3,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1973,0.00,1.60,32.601011,-86.680736,1970
4,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1974,0.00,1.60,32.601011,-86.680736,1970
...,...,...,...,...,...,...,...,...,...,...
2596,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2016,5.15,7.25,43.000325,-107.554567,2010
2597,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2017,5.15,7.25,43.000325,-107.554567,2010
2598,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2018,5.15,7.25,43.000325,-107.554567,2010
2599,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2019,5.15,7.25,43.000325,-107.554567,2010


In [37]:
geojson_data_final=geojson_data.merge(statistics_df, on=['Decade','State'])
geojson_data_final

,id,State,density,geometry,Year,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude,Decade,mean,median
0,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1970,0.00,1.30,32.601011,-86.680736,1970,0.00,0.00
1,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1971,0.00,1.30,32.601011,-86.680736,1970,0.00,0.00
2,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1972,0.00,1.60,32.601011,-86.680736,1970,0.00,0.00
3,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1973,0.00,1.60,32.601011,-86.680736,1970,0.00,0.00
4,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1974,0.00,1.60,32.601011,-86.680736,1970,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2596,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2016,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15
2597,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2017,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15
2598,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2018,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15
2599,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2019,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15


In [38]:
geojson_data_final = geojson_data_final.merge(statistics_df_federal, on=['Decade','State'])
geojson_data_final

,id,State,density,geometry,Year,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude,Decade,mean_x,median_x,mean_y,median_y
0,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1970,0.00,1.30,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
1,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1971,0.00,1.30,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
2,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1972,0.00,1.60,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
3,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1973,0.00,1.60,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
4,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1974,0.00,1.60,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2596,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2016,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2597,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2017,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2598,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2018,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2599,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2019,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25


In [39]:
geojson_data_to_upload=geojson_data_final.rename(columns = {'mean_x':'State_Average_by_Decade', 'median_x':'State_Median_by_Decade','mean_y':'Federal_Average_by_Decade', 'median_y':'Federal_Median_by_Decade'})
geojson_data_to_upload

,id,State,density,geometry,Year,State.Minimum.Wage,Federal.Minimum.Wage,Latitude,Longitude,Decade,State_Average_by_Decade,State_Median_by_Decade,Federal_Average_by_Decade,Federal_Median_by_Decade
0,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1970,0.00,1.30,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
1,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1971,0.00,1.30,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
2,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1972,0.00,1.60,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
3,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1973,0.00,1.60,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
4,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",1974,0.00,1.60,32.601011,-86.680736,1970,0.00,0.00,1.85,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2596,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2016,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2597,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2017,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2598,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2018,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25
2599,56,Wyoming,5.851,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",2019,5.15,7.25,43.000325,-107.554567,2010,5.15,5.15,7.25,7.25


In [40]:
geojson_data_to_upload.to_file("minwage2try.geojson", driver='GeoJSON')